=====DATSCIW261 ASSIGNMENT #3=====

MIDS UC Berkeley, Machine Learning at Scale

DATSCIW261 ASSIGNMENT #3

Version 2016-01-27 (FINAL)

#### Hetal Chandaria (hetalchandaria@berkeley.edu)
W261 - 2 , ASSIGNMENT #3

Submission Date : add

#### HW3.0.
What is a merge sort? Where is it used in Hadoop?

How is  a combiner function in the context of Hadoop?  Give an example where it can be used and justify why it should be used in the context of this problem.

What is the Hadoop shuffle?

### <span style="color:blue;">Answer</span>

#### Merge Sort 

Merge sort is a sort algorithm that sorts a dataset by divide-and-conquer. The data set is split continuously into sub-sets until each sub-set is size of 1 and therefore considered automatically sorted. Now, sub-sets are merged together to form a final fully sorted data-set. Given n sorted-subsets, the complexity to merge these sorted sub-sets is proportional to the total no. of elements in all the sub-sets combined.

Hadoop can be considered in some sense an implementation of merge sort as it first partitions the dataset into multiple map tasks, sorts each map output and eventually does a "merge" of each map's sorted output in the reducer. A lot of the heavy lifting of this implementation is done in the shuffle layer. 

#### Combiner Function

Given that data comes to the mapper in no defined order, the mapper output can be quite verbose as it can potentially not make any optimizations. The combiner function is run on mapper output and can "coalesce/combine" the mapper output so that the verbose map output does not fully need to be sent to the reducer. The combiner pretty much does the same work as that of the reducer. It has to process data in the format of the mapper output but unlike the reducer, it also generate output that is consumable by the reducer. 

Let us consider a job that accepts the following input (word, priority) and the aim is to generate the avg. priority for each word. For a job without a combiner, the mapper could generate the following output (word, priority). This would be then averaged upon by the reducer. However, let's say that the word "foo" occurs a million times in the same mapper. In this scenario, without a combiner, 1 million rows will be sent to the reducer. A combiner if run could potentially optimize this into an key-value output such as (word, (sum(priority), frequency of word)). Using this tuple, the reducer can then calculate the avg. priority. Note, that the mapper output would also change to (word, (priority, 1)).

Not all operations support a combiner. Only commutative and associative operations can work as the output of the reducer has to be the same regardless of whether the combiner ran or not. 

#### Hadoop shuffle

The hadoop shuffle is the layer that transfers the data from the mappers to the reducers. Consider 4 maps and 2 reducers. Each mapper will generate data in 2 partitions considering that there are 2 reducers. This is governed by the partitioner function which can be overridden by the user. The shuffle layer will try to efficiently transfer the data from each mapper to a particular reducer, ensure that the data is sorted correctly by the configured keys ( by defining an approrpiate comparator used to compare keys when sorting ) and that data for a given key is provided in a sorted/grouped manner to the reducer. This is what enables us to write simple reducers without requiring us to do a lot of in-memory buffering. For example, consider the input data of format (word, priority) where the aim is to find max(priority) for each word. If the words came to the reducer in a random order, the reducer would need to keep a lookup table to track each word and its max priority given that there is no information on whether the word may or may not show up later. Given the sorted nature of input to the reducer, the simplistic reducer knows that as soon as a new word is seen, the previous word will never again show up and the max priority can then be generated for that word. This means no need to have a large lookup table for each seen word but rather just one object to track the data for the current word and its max priority.

The diagram below depicts the various components of hadoop shuffle. 

<img src="./images/shuffle.jpg">



In [ ]:
#### Create directories in HDFS for HW3
!hdfs dfs -mkdir hw3
!hdfs dfs -mkdir hw3/src
!hdfs dfs -mkdir hw3/output

#### HW3.1 
Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,

1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,

1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,


User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible.

In [65]:
%%writefile mapper.py
#!/usr/bin/python

#HW3.1 In this question ,we will only have a mapper which emit counters based on the type of complaint.

import sys
import re
from csv import reader



# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    key = None
    if(token[1]=='Debt collection'):
        sys.stderr.write("reporter:counter:Debt-counter,debt,1\n")
    elif(token[1]=='Mortgage'):
        sys.stderr.write("reporter:counter:Mortgage-counter,mortgage,1\n")
    else:
        sys.stderr.write("reporter:counter:Other-counter,other,1\n")

Overwriting mapper.py


In [66]:
# function to run the hadoop job 
def hw3_1():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw31
    
    #remove header line from Consumer_Complaints.csv
    !tail -n $(($(wc -l < Consumer_Complaints.csv) - 1))  < Consumer_Complaints.csv > Consumer_Complaints_mod.csv
    
    #move input file to hdfs 
    !hdfs dfs -rm hw3/src/Consumer_Complaints_mod.csv
    !hdfs dfs -put Consumer_Complaints_mod.csv  hw3/src/
    
    # run map reduce job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar \
    -Dmapred.reduce.tasks=0 \
    -file mapper.py \
    -mapper mapper.py \
    -input hw3/src/Consumer_Complaints_mod.csv \
    -output hw3/output/hw31 \

hw3_1()

16/01/31 16:18:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw31
16/01/31 16:18:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/src/Consumer_Complaints_mod.csv
16/01/31 16:18:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 16:18:39 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 16:18:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar6525908040360619174/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob381896063830971794.jar tmpDir=null


#### Output of the above from job tracker

<img src="./images/hw31.jpg">

#### HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [70]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.2a In this question, we will emit a counter for everytime the mapper is called. 
# output of mapper is word and 1
import sys
import re

sys.stderr.write("reporter:counter:HW_32a,num_mappers,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    for token in line.strip().split(" "):
        print '%s,%s' % (token, 1)

Overwriting mapper.py


In [71]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.2a In this question, we will emit a counter for everytime the reducer is called. 

import sys

sys.stderr.write("reporter:counter:HW_32a,num_reducers,1\n")
last_key = None
word = None
total_count = 0
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split(',', 1)
    
    #if current key is same as last_key than increment count 
    if(last_key == word):
        total_count += int(count)
    else:
        if (last_key):
            print '%s\t%s' %(last_key,total_count)
        total_count = int(count)
        last_key = word
if last_key == word:
    print '%s\t%s' %(last_key,total_count)

Overwriting reducer.py


In [72]:
# function to run the hadoop job 
def hw3_2_a():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #create inpur file and move it to hdfs 
    !echo "foo foo quux labs foo bar quux" > hw32_a_input.txt 
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw32a
    
    #move input file to hdfs 
    !hdfs dfs -rm hw3/src/hw32_a_input.txt
    !hdfs dfs -put hw32_a_input.txt  hw3/src/
    
    # run map reduce job. Here we have explicitly set the number of mappers and reducers
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar \
    -Dmapred.map.tasks=1 \
    -Dmapred.reduce.tasks=4 \
    -file mapper.py \
    -mapper mapper.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/hw32_a_input.txt \
    -output hw3/output/hw32a
    
    print "\n"
    !echo "Word and its frequency"
    !hdfs dfs -cat hw3/output/hw32a/part*

hw3_2_a()

16/01/31 16:22:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw32a
16/01/31 16:22:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/src/hw32_a_input.txt
16/01/31 16:22:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 16:22:48 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 16:22:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar3923939505159964458/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob6363776760723693083.jar tmpDir=null


Word and its fr

#### Output for first part

<img src="./images/hw32a.jpg">

#### 3.2.b

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers). Perform a word count analysis of the Issue column of the Consumer Complaints Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.

In [73]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.2b In this question, we will emit a counter for everytime the mapper is called. 
# output of mapper is issue which is token[3] and 1

import sys
from csv import reader


sys.stderr.write("reporter:counter:HW_32b,num_mappers,1\n")

# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    #emit only 3rd token as it contains text about issue
    print '"%s",%s' %(token[3],1)

Overwriting mapper.py


In [74]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.2b In this question, we will emit a counter for everytime the reducer is called. 

import sys
from csv import reader

sys.stderr.write("reporter:counter:HW_32b,num_reducers,1\n")
last_key = None #user for keeping track of which key is currently being aggregated
word = None
total_count = 0 #counter to increment frequency
# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    word=token[0]
    count = int(token[1])
    
    #if current key is same as last_key than increment count 
    if(last_key == word):
        total_count += int(count)
    else:
        if (last_key):
            print '%s\t%s' %(last_key,total_count)
        total_count = int(count)
        last_key = word
if last_key == word:
    print '%s\t%s' %(last_key,total_count)

Overwriting reducer.py


In [75]:
# function to run the hadoop job 
def hw3_2_b():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw32b
    
    # run map reduce job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar \
    -Dmapred.map.tasks=2 \
    -Dmapred.reduce.tasks=2 \
    -file mapper.py \
    -mapper mapper.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/Consumer_Complaints_mod.csv \
    -output hw3/output/hw32b
    
    print "\n"
    !echo "HDFS Output"
    !hdfs dfs -cat hw3/output/hw32b/part*

hw3_2_b()

16/01/31 16:26:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw32b
16/01/31 16:26:45 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 16:26:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar6961344658254863619/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob5728141958360488655.jar tmpDir=null


HDFS Output
16/01/31 16:27:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
APR or interest rate	3431
Account opening, closing, or management	16205
Account terms and changes	350
Application processing delay	243
Applied for loan/did not receive money	139
Billin

#### Output for 32b of custome counters
<img src="./images/hw32b.jpg">

#### 3.2.c

Perform a word count analysis of the Issue column of the Consumer Complaints Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).

In [76]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.2c In this question, we will emit a counter for everytime the mapper is called. 
# output of mapper is issue which is token[3] and 1
import sys
from csv import reader

sys.stderr.write("reporter:counter:HW_32c,num_mappers,1\n")

# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    #emit issue related token
    print '"%s",%s' %(token[3],1)
    


Overwriting mapper.py


In [78]:
%%writefile combiner.py
#!/usr/bin/python
#HW3.2c In this question, we will emit a counter for everytime the combiner is called. 
#the combiner will do intermediate aggregation of data and is similar to reducer in terms of logic
import sys
from csv import reader

sys.stderr.write("reporter:counter:HW_32c,num_combiners,1\n")
last_key = None
word = None
total_count = 0
# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    word=token[0]
    count = int(token[1])
    
    #if current key is same as last_key than increment count 
    if(last_key == word):
        total_count += int(count)
    else:
        if (last_key):
            print '"%s",%s' %(last_key,total_count)
        total_count = int(count)
        last_key = word
if last_key == word:
    print '"%s",%s' %(last_key,total_count)

Overwriting combiner.py


In [79]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.2c In this question, we will emit a counter for everytime the reducer is called. 

import sys
from csv import reader

sys.stderr.write("reporter:counter:HW_32c,num_reducers,1\n")
last_key = None
word = None
total_count = 0
# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    word=token[0]
    count = int(token[1])
    
    #if current key is same as last_key than increment count 
    if(last_key == word):
        total_count += int(count)
    else:
        if (last_key):
            print '"%s",%s' %(last_key,total_count)
        total_count = int(count)
        last_key = word
# do not forget to output the last word
if last_key == word:
    print '"%s",%s' %(last_key,total_count)

Overwriting reducer.py


In [80]:
# function to run the hadoop job 
def hw3_2_c():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw32c
    
    # run map reduce job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar \
    -Dmapred.map.tasks=4 \
    -Dmapred.reduce.tasks=2 \
    -file mapper.py \
    -mapper mapper.py \
    -file combiner.py \
    -combiner combiner.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/Consumer_Complaints_mod.csv \
    -output hw3/output/hw32c
    
    print "\n"
    !echo "HDFS Output"
    !hdfs dfs -cat hw3/output/hw32c/part*

hw3_2_c()

16/01/31 16:32:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw32c
16/01/31 16:32:53 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 16:32:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, combiner.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar1327447845199348294/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob3379068482288296690.jar tmpDir=null


HDFS Output
16/01/31 16:33:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"APR or interest rate",3431	
"Account opening, closing, or management",16205	
"Account terms and changes",350	
"Application processing delay",243	
"Applied for loan/did no

#### Output of mapper, recucer and combiner counters
<img src="./images/hw32c.jpg">

####  part 2 of 3_2_c
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).

In [83]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.2c_1 In this question, we will emit a counter for everytime the mapper is called. 
# output of mapper is frequency and issue 
# output of previous mapreduce job is used as input for this map reduce task
import sys
from csv import reader

sys.stderr.write("reporter:counter:Mapper-counter,num_mappers,1\n")

# input comes from STDIN (standard input)
for token in reader(sys.stdin):
    print '%s\t"%s"' %(token[1],token[0])

Overwriting mapper.py


In [86]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.2c_1 In this question, we will emit a counter for everytime the reducer is called. 
# In this case of mapreduce we are using secondary sort where in we first sort on the frequency (ascending) 
# and than sort on the issue in case of tie

import sys, Queue
import re
from csv import reader

sys.stderr.write("reporter:counter:Reducer-counter,num_reducers,1\n")

n_max = 50 # top n issues 
n_min = 10 # bottom n issues 
a_min = [] # list to hold the bototom issues 
q_max = Queue.Queue(n_max) # Queue to hold the top 50 issues
total_count =0 

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    freq = int(row[0])
    key = row[1]
    #increment the count
    total_count += freq
    
    # add the lowest 10 words as we are getting data in sorted order already
    if len(a_min) < n_min:
        a_min.append((key,freq))
    
    #Now add the top 10 words . In this case we use queue as its FIFO which will always pop the lowest element
    if q_max.full():
        q_max.get()
    q_max.put((key,freq))
    
print '\n Output = Word , Frequency and Relative Frequency'

print '\n%d Bottom issues:' %n_min
for record in a_min:
    print record[0] +"\t"+str(record[1])+"\t"+str(1.0*record[1]/total_count)

print '\n%d Top issues:' %n_max
for i in range(n_max):
    record = q_max.get()
    print record[0] +"\t"+str(record[1])+"\t"+str(1.0*record[1]/total_count)

Overwriting reducer.py


In [87]:
# function to run the hadoop job 
def hw3_2_c():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw32c_2
    
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options="-k1,1n -k2,2" \
    -D mapred.map.tasks=4 \
    -D mapred.reduce.tasks=1 \
    -mapper mapper.py \
    -file mapper.py \
    -reducer reducer.py \
    -file reducer.py \
    -input hw3/output/hw32c/part* \
    -output hw3/output/hw32c_2
    
    
    print "\n"
    !echo "HDFS Output"
    !hdfs dfs -cat hw3/output/hw32c_2/part*

hw3_2_c()

16/01/31 16:40:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw32c_2
16/01/31 16:40:04 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 16:40:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar8680464347089963391/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob7457856801864801774.jar tmpDir=null


HDFS Output
16/01/31 16:40:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
	
 Output = Word , Frequency and Relative Frequency	
	
10 Bottom issues:	
"Incorrect/missing disclosures or info"	64	0.000204532961765
"Charged bank acct wrong day or amt"	71	0.00022

#### HW 3.2.1 OPTIONAL 
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). Please use a combiner.

#### HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [6]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.3 In this question, we will emit a counter for everytime the reducer is called. 
# Here the mapper reads each session information and emits product along with the count.
# We also emit the number of products in each basket and basket count as for each line read.
import sys
import re

sys.stderr.write("reporter:counter:HW3_3,num_mapper,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    count = 0 #used to determine basket length
    # remove leading and trailing whitespace
    for token in line.strip().split(" "):
        print '%s\t%s' % (token, 1)
        count += 1
    print '%s\t%s'%('*BASKET',count) # total number of products in the basket
    print '%s\t%s'%('*NUM_OF_BASKET',1) # basket count 

Overwriting mapper.py


In [7]:
%%writefile combiner.py
#!/usr/bin/python
#HW3.3 In this question, we will emit a counter for everytime the combiner is called. 
# Here the combiner code does intermediate aggregation for product, number of baskets 
import sys
import re

sys.stderr.write("reporter:counter:HW3_3,num_combiners,1\n")

last_key = None
word = None
total_count = 0
max_basket = -1
basket_cnt = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    product = row[0]
    count = int(row[1])
    
    #if its basket count determine which whether the basket is largest so far and continue
    if (product =='*BASKET'):
        if(count > max_basket):
            max_basket = count
        continue
    #if the data is about number of baskets increment the count and continue
    elif (product =='*NUM_OF_BASKET'):
        basket_cnt += count
        continue
    #else aggregate the product data
    else:    
        #if current key is same as last_key than increment count 
        if(last_key == product):
            total_count += count
        else:
            if (last_key):
                print '%s\t%s' %(last_key,total_count)
            total_count = count
            last_key = product
if last_key == product:
    print '%s\t%s' %(last_key,total_count)

if(max_basket != -1):
    print '%s\t%s' %('*BASKET',max_basket) 
print '%s\t%s' %('*NUM_OF_BASKET',basket_cnt) 


Overwriting combiner.py


In [11]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.3 In this question, we will emit a counter for everytime the reducer is called. 
#here we have used heapq to store the 50 top products frequently purchased. 

import sys
import re
import heapq

sys.stderr.write("reporter:counter:HW3_3,num_reducers,1\n")

last_key = None
word = None
total_count = 0
max_basket = -1
n_max = 100 #top n products purchased
unique_cnt = 0 #number of unique products 
max_queue =  [] # hold top n products 
num_baskets = 0

#function to add to queue.
#we have used heapq here so we can remove the smallest element if the queue is full during insertion.
def add_to_max_Q(max_q,product,cnt):
    # add elements if q is not full
    if (len(max_q) < n_max):
        heapq.heappush(max_q, (cnt, product))
    else:
        # add new element and then remove smallest element
        heapq.heappush(max_q, (cnt, product))
        heapq.heappop(max_q)

#function to print the queue data
def print_topn(max_q,total_bsk):
    cnt = len(max_q)
    if(cnt > n_max):
        cnt = n_max
    s = heapq.nlargest(cnt, max_q)
    for row in s:
        print '%s\t%s\t%s'%(row[1] ,row[0],(1.0*row[0]/total_bsk))
    

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    product = row[0]
    count = int(row[1])
    
    #if data is about basket length, determine which is largest and continue
    if (product =='*BASKET'):
        if(count > max_basket):
            max_basket = count
        continue
    #if data is about number of baskets , than increment total count
    elif (product == '*NUM_OF_BASKET'):
        num_baskets += count
        continue
    #aggregate products 
    else:    
        #if current key is same as last_key than increment count 
        if(last_key == product):
            total_count += count
        else:
            if (last_key):
                #increment unique product count and add to our queue 
                unique_cnt += 1
                add_to_max_Q(max_queue,last_key,total_count) 
            total_count = count
            last_key = product
if last_key == product:
    unique_cnt += 1
    #uncomment below line if you want use heapq
    add_to_max_Q(max_queue,last_key,total_count)

    
print '%s\t%s\t%s' %('*LARGEST_BASKET',max_basket,1)
print '%s\t%s\t%s' %('*UNIQUE_CNT',unique_cnt,1)
print_topn(max_queue,num_baskets)

# print "\n Largest Basket : %s" %(max_basket)
# print "\n Number of unique products: %s" %(unique_cnt)
# print "\n Top 50 products (product, frequency, relative frequency )\n "


Overwriting reducer.py


In [12]:
# function to run the hadoop job 
def hw3_3():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw33
    
    #move input file to hdfs 
    !hdfs dfs -rm hw3/src/ProductPurchaseData.txt
    !hdfs dfs -put ProductPurchaseData.txt  hw3/src/
    
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -file mapper.py \
    -mapper mapper.py \
    -file combiner.py \
    -combiner combiner.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/ProductPurchaseData.txt \
    -output hw3/output/hw33
    
#     print "\n"
#     !echo "HDFS Output"
#     !hdfs dfs -cat hw3/output/hw33/part*

hw3_3()

16/01/31 22:06:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw33
16/01/31 22:06:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/src/ProductPurchaseData.txt
16/01/31 22:06:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 22:06:16 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 22:06:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, combiner.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar1866844824180613390/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob2321349398462098251.jar tmpDir=nul

In [14]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.3 Writing a second map reduce job to do sorting on the output from the previous job
import sys
import re

sys.stderr.write("reporter:counter:HW3_3b,num_mapper,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    token = line.strip().split('\t')
    print '%s\t%s\t%s' %(token[1],token[0],token[2])

Overwriting mapper.py


In [15]:
%%writefile reducer.py
#!/usr/bin/python
#HW3.3 Writing a second  reduce job to print the 50 top products 
import sys
import re

sys.stderr.write("reporter:counter:HW3_3b,num_mapper,1\n")

print "\n Top 50 products (product, frequency, relative frequency )\n "
counter = 0
max_basket=0
unique_cnt =0
n_max=50

# input comes from STDIN (standard input)
for line in sys.stdin:
    token = re.split(r'\t+',line.strip())
    
    if(token[1]=='*LARGEST_BASKET'):
        max_basket = token[0]
        continue
    elif(token[1]=='*UNIQUE_CNT'):
        unique_cnt = token[0]
        continue;
    else:
        if(counter<n_max):
            print '%s\t%s\t%s'%(token[1] ,token[0],token[2]) 
            counter += 1

print "\n Largest Basket : %s" %(max_basket)
print "\n Number of unique products: %s" %(unique_cnt)    

Overwriting reducer.py


In [16]:
# function to run the hadoop job 
def hw3_3_b():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw33b
    
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options="-k1,1nr -k2,2" \
    -D mapred.reduce.tasks=1 \
    -mapper mapper.py \
    -file mapper.py \
    -reducer reducer.py \
    -file reducer.py \
    -input hw3/output/hw33/part* \
    -output hw3/output/hw33b
    
    
    print "\n"
    !echo "HDFS Output"
    !hdfs dfs -cat hw3/output/hw33b/part*

hw3_3_b()

16/01/31 22:06:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw33b
16/01/31 22:06:49 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 22:06:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar5123181696967325995/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob6697234280007067738.jar tmpDir=null


HDFS Output
16/01/31 22:07:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
	
 Top 50 products (product, frequency, relative frequency )	
 	
DAI62779	6667	0.214366097553
FRO40251	3881	0.124786984341
ELE17451	3875	0.1245940645
GRO73461	3602	0.115816211697
SNA80

#### 3.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce.

#### HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [17]:
%%writefile mapper.py
#!/usr/bin/python
#HW3.4 In this question, we will emit a counter for everytime the mapper is called. 
# In this mapper code, we are first taking unique products for each basket ( if there are duplicates ) 
# and than sorting it. We have used python function of combinations to emit all tuples of length 2.
# We also emit for everyline read a count for the number of baskets 

import sys
import re
import itertools

sys.stderr.write("reporter:counter:HW3_4,num_mapper,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # emit count for basket
    print '%s\t%s\t%s'%('*','BASKET',1)
    
    # remove leading and trailing whitespace and tokenize
    token  =  line.strip().split(" ")
    for subset in itertools.combinations(sorted(set(token)), 2):
        print '%s\t%s\t%s' % (subset[0],subset[1], 1)
    

Overwriting mapper.py


In [18]:
%%writefile combiner.py
#!/usr/bin/python

#HW3.4 In this question, we will emit a counter for everytime the combiner is called. 
# In this combiner code we aggregate the basket count and products that occur together.
# This is done for intermediate aggregation

import sys
import re


sys.stderr.write("reporter:counter:HW3_4,num_combiner,1\n")
last_key = None
word = None
total_count = 0
basket_cnt = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    product = (row[0],row[1]) #combine product1 and product2 as tuple to be used as key
    count = int(row[2])
    #increment total number of baskets if incoming data is for baskets 
    if (product ==('*','BASKET')):
        basket_cnt += count
        continue

    else:    
        #if current key is same as last_key than increment count 
        if(last_key == product):
            total_count += count
        else:
            if (last_key):
                print '%s\t%s\t%s' %(last_key[0],last_key[1],total_count)
            total_count = count
            last_key = product
if last_key == product:
    print '%s\t%s\t%s' %(last_key[0],last_key[1],total_count)

print '%s\t%s\t%s' %('*','BASKET',basket_cnt) 

Overwriting combiner.py


In [19]:
%%writefile reducer.py
#!/usr/bin/python

#HW3.4 In this question, we will emit a counter for everytime the reducer is called. 
# In this reducer code we aggregate the basket count and products that occur together.

import sys
import re


sys.stderr.write("reporter:counter:HW3_4,num_reducers,1\n")
last_key = None
word = None
total_count = 0
max_basket = -1
basket_cnt = 0
sup_cnt = 100 
# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    product = (row[0],row[1]) #combine product1 and product2 as tuple to be used as key
    count = int(row[2])
    #increment total number of baskets if incoming data is for baskets 
    if (product ==('*','BASKET')):
        basket_cnt += count
        continue
    #increment the count for products that occur together
    else:    
        #if current key is same as last_key than increment count 
        if(last_key == product):
            total_count += count
        else:
            #emit pair of product if they are greater than 100
            if (last_key and total_count > sup_cnt):
                print '%s\t%s\t%s' %(last_key[0],last_key[1],total_count)
            total_count = count
            last_key = product
if (last_key == product and total_count > sup_cnt ):
    print '%s\t%s\t%s' %(last_key[0],last_key[1],total_count)
#emit the total number of baskets
print '%s\t%s\t%s' %('*','NUM_OF_BASKET',basket_cnt)

Overwriting reducer.py


In [21]:
import time

# function to run the hadoop job 
def hw3_4a():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw34a
    
    start = time.time()
    
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
    -D mapred.reduce.tasks=1 \
    -file mapper.py \
    -mapper mapper.py \
    -file combiner.py \
    -combiner combiner.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/ProductPurchaseData.txt \
    -output hw3/output/hw34a
     
    print"\n Time taken to run this section of code in seconds: "
    print time.time() - start
#     print "\n"
#     !echo "HDFS Output"
#     !hdfs dfs -cat hw3/output/hw34a/part*

hw3_4a()

16/01/31 22:11:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw34a
16/01/31 22:11:43 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 22:11:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, combiner.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar4466300746263274902/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob4216056481942220430.jar tmpDir=null

 Time taken to run this section of code in seconds: 
31.7277860641


#### Implementing second mapreduce job to determine top 50 product pairs which coccur 

In [22]:
%%writefile mapper.py
#!/usr/bin/python

#HW3.4 In this question, we will emit a counter for everytime the mapper is called is called. 
# In the mapper code we will emit the frequency first followed by the product pair
# we are also going to tell hadoop to sort the key as numeric and in reverse order

import sys
import re

sys.stderr.write("reporter:counter:HW3_4b,num_mappers,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    #remove leading and trailing spaces 
    row = re.split(r'\t',line.strip())
    print '%s\t%s\t%s' %(row[2],row[0],row[1])


Overwriting mapper.py


In [23]:
%%writefile reducer.py
#!/usr/bin/python

#HW3.4 In this question, we will emit a counter for everytime the reducer is called is called. 
# As we are sorting the data in descending order based on the frequency, we will get the number of baskets on top.
# after that we enter the top 50 product pair in the list and print the record
import sys
import re

sys.stderr.write("reporter:counter:HW3_4b,num_reducers,1\n")

n_max = 50
a_max = []
num_baskets =0 

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t+',line.strip())
    freq = int(row[0])
    key = (row[1],row[2])
    
    if(key==('*','NUM_OF_BASKET')):
        num_baskets = freq
        continue
    else:
        # add the lowest 10 words 
        if len(a_max) < n_max:
            a_max.append((key,freq,1.0*freq/num_baskets))
        
for record in a_max:
    print record

Overwriting reducer.py


In [25]:
# function to run the hadoop job 
import time 
def hw3_4_b():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw34b
    
    start = time.time()
    
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=3 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.partition.keycomparator.options="-k1,1nr -k2,2 -k3,3" \
    -D mapred.reduce.tasks=1 \
    -mapper mapper.py \
    -file mapper.py \
    -reducer reducer.py \
    -file reducer.py \
    -input hw3/output/hw34a/part* \
    -output hw3/output/hw34b
    
    print"\n Time taken to run this section of code in seconds: "
    print time.time() - start
    
    print "\n"
    !echo "HDFS Output"
    !hdfs dfs -cat hw3/output/hw34b/part*

hw3_4_b()

16/01/31 22:14:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw34b
16/01/31 22:14:27 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 22:14:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar759612420876421916/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob9181965335011797423.jar tmpDir=null

 Time taken to run this section of code in seconds: 
16.7769880295


HDFS Output
16/01/31 22:14:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
(('DAI62779', 'ELE17451'), 902, 0.0290022828847947)	
(('FRO40251', 'GRO85051'), 859, 0.027619690685186972)	
(('FRO402

#### Computational setup used 

1. Single node haddop cluster , running hadoop version 2.7.1. This was setup on my mac (intel i7, 4 cores)
2. Yarn is enabled and all mapreduce jobs are running in yarn mode
3. Yarn configured with a single node manager / slave. This node manager has 8GB capacity. Yarn vcore /cpu support was not enabled 
4. Each map task is configured to use 1 GB resource (mapreduce.map.memory.mb = 1024, default value)
5. Each reduce task is configured to use 1 GB resource (mapreduce.reduce.memory.mb = 1024, default value)
6. Given this, it means that my yarn cluster could run at max 6 maps/reduce tasks concurrently (yarn.app.mapreduce.am.resource.mb = 1536, MR application master is using 2GB)  

Total time taken = 
From Job a : 
Time Taken : 31.7277860641 secs
Number of mapper = 2
Number of reducers = 1
Number of combiners = 2

From Job b : 
Time Taken : 16.7769880295 secs
Number of mapper = 2
Number of reducers = 1
Number of combiners = 0

Image from Job a
<img src="./images/hw34a.jpg">

Image from Job b
<img src="./images/hw34b.jpg">

#### HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs

In [115]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

sys.stderr.write("reporter:counter:HW3_5a,num_mappers,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # Emit * , BAKSET for counting total number of baskets 
    print '%s\t%s\t%s'%('*','BASKET',1)
    # remove leading and trailing whitespace and tokenize
    token  =  line.strip().split(" ")
    uniq_tokens = sorted(set(token))
    for i in range (0,len(uniq_tokens)-1):
        key = uniq_tokens[i]
        value = ""
        for j in range(i+1, len(uniq_tokens)):
            value += uniq_tokens[j] + "\t1\t"
        print '%s\t%s' %(key,value)

Overwriting mapper.py


In [147]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import heapq
import operator

sys.stderr.write("reporter:counter:HW3_5a,num_reducers,1\n")

last_key = None
word = None
total_count = 0
max_basket = -1
basket_cnt = 0 
sup_cnt = 100 # variable to hold the minimum support count 
tmp_dict ={} # for aggregation
n_max = 50 # top n product pairs 
max_q = []

#function to add to queue.
#we have used heapq here so we can remove the smallest element if the queue is full during insertion.
def add_to_max_Q(p1,p2,cnt):
    # add elements if q is not full
    if (len(max_q) < n_max):
        heapq.heappush(max_q, (cnt, (p1,p2)))
    else:
        # add new element and then remove smallest element
        heapq.heappush(max_q, (cnt, (p1,p2)))
        heapq.heappop(max_q)
    
#function to print the queue data
def print_topn(total_bsk):
    cnt = len(max_q)
    if(cnt > n_max):
        cnt = n_max
    s = heapq.nlargest(cnt, max_q)
    for row in s:
        print '%s\t%s\t%s\t%s'%(row[1][0] ,row[1][1],row[0],(1.0*row[0]/total_bsk))

def merge_dict(merged,row):
    for product in row:
        if product in merged:
            merged[product] += row[product]
        else:
            merged[product] = row[product]
    return merged

def emit_dict(key,merged):
    output = key
    cnt =0
    merged_sorted = sorted(merged.items(), key=operator.itemgetter(0))
    for tup in merged_sorted:
        # for prd, value in merged_sort.iteritems():
        prd = tup[0]
        value = tup[1]
        if(value < sup_cnt):
            continue
        cnt += 1
        add_to_max_Q(key,prd,value)
        output += "\t" + prd + "\t" + str(value)

# input comes from STDIN (standard input)
for line in sys.stdin:
    row = re.split(r'\t',line.strip())
    product = row[0]
    #increment the total basket if we encounter * BASKET tuple
    if(row[0]=='*' and row[1]=='BASKET'):
        basket_cnt += int(row[2])
        
        continue
        
    #increment the count for products that occur together. 
    else:
        row_dict ={}
        #parse the remaining data elements and increment counters by 2
        for x in range (1,len(row),2):
            prd = row[x]
            prd_cnt = int(row[x+1])
            row_dict[prd]=prd_cnt
        
        if(last_key == product):
            #update dictionary                
            tmp_dict=merge_dict(tmp_dict,row_dict)

        #emit the data is we find new product key 
        else:
            #emit pair of product if they are greater than 100
            if (last_key):
                #print dictionary
                emit_dict(last_key,tmp_dict)
            #create dictonary
            tmp_dict=row_dict
            last_key = product
                        
            
if (last_key == product):
    #print dictionary
    emit_dict(last_key,tmp_dict)
    
#emit the total number of baskets
print '%s\t%s\t%s' %('*','NUM_OF_BASKET',basket_cnt)     
print "\n %d Top product pairs that occur together" %(n_max)
print_topn(basket_cnt)

Overwriting reducer.py


In [149]:
import time
# function to run the hadoop job 
def hw3_5a():
    
    #change properties of mapper.py 
    !chmod a+x mapper.py;chmod a+x reducer.py
    
    #remove output directory if present else hadoop job gives error
    !hdfs dfs -rm -r hw3/output/hw35a
    
    start = time.time()
    #run hadoop job
    !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-*streaming*.jar  \
    -file mapper.py \
    -mapper mapper.py \
    -file reducer.py \
    -reducer reducer.py \
    -input hw3/src/ProductPurchaseData.txt \
    -output hw3/output/hw35a
    
    print"\n Time taken to run this section of code in seconds: "
    print time.time() - start
    print "\n"
    !hdfs dfs -cat hw3/output/hw35a/part*
    
hw3_5a()

16/01/31 19:10:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3/output/hw35a
16/01/31 19:10:16 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 19:10:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/hadoop-unjar839828248141599493/] [] /var/folders/91/cjfxt7ys6958qll6vjtgwwfw0000gn/T/streamjob4969713235079721198.jar tmpDir=null

 Time taken to run this section of code in seconds: 
22.3631811142


16/01/31 19:10:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
*	NUM_OF_BASKET	31101
	
 50 Top product pairs that occur together	
DAI62779	ELE17451	1592	0.0511880646925
FRO40251	SNA80324	1412	

In [ ]:
%%writefile mapper.py
#!/usr/bin/python

#hw3.5b 

import sys
import re

sys.stderr.write("reporter:counter:HW3_5b,num_mappers,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace and tokenize
    token  =  line.strip().split("\t")
    for i in range (0,len(uniq_tokens)-1):
        key = uniq_tokens[i]
        value = ""
        for j in range(i+1, len(uniq_tokens)):
            value += uniq_tokens[j] + "\t1\t"
        print '%s\t%s' %(key,value)
   

In [150]:
%%writefile reducer.py
#!/usr/bin/python

#hw3.5b 

import sys
import re

sys.stderr.write("reporter:counter:HW3_5b,num_reducers,1\n")
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace and tokenize
    token  =  line.strip().split("\t")
    for i in range (0,len(uniq_tokens)-1):
        key = uniq_tokens[i]
        value = ""
        for j in range(i+1, len(uniq_tokens)):
            value += uniq_tokens[j] + "\t1\t"
        print '%s\t%s' %(key,value)

Overwriting mapper.py
